# Import the Python libraries we can use by default:

In [ ]:
import numpy as np
from collections import deque

# Definition for nodes, edges, and coordinates:

In [ ]:
# Class to support coordinates:
class coordinate:
  def __init__(self, x, y):
    self.x = x
    self.y = y

# Class to support nodes:
class node:
    def __init__(self, id, coordinate):
      self.id = id
      self.coordinate = coordinate
      self.visited = False
      self.hValue = None
      self.gValue = np.inf

# Class to support edges:
class edge:
    def __init__(self, destination, weight):
        self.destination = destination
        self.weight = weight

# Global variable definitions

In [ ]:
# Costly in memory as nodes go up, but helps immensely in quick access of between edge and node array
nodeIDToCoordinateDict = {}

# Global variables for starting and ending node
startNode = None
endNode = None

# Main function:

In [ ]:
def main():
  # Get the user's test case, find the nodes from it, find the edges from it.
  userTestCase = getTestCase()
  nodeList = generateNodeList(userTestCase)
  edgeDict = generateEdgeList(userTestCase)

  # Run BFS, DFS, and A* algorithms
  BFSVisitedList = BFS(nodeList, edgeDict)
  print("BFS: ", end='')
  print(BFSVisitedList)

  resetAllNodes(nodeList)

  DFSVisitedList = DFS(nodeList, edgeDict)
  print("DFS: ", end='')
  print(DFSVisitedList)

  resetAllNodes(nodeList)

  # A* has 3 different heuristics. Heuristic will be accepted as input in
  # form of a number. Hard coded number but can be abstracted to a constant
  # if needed.
  for heuristicNum in range(1, 4):
    AStarVisitedList = AStar(nodeList, edgeDict, heuristicNum)
    print("A* with Heuristic " + str(heuristicNum) + ": ", end='')
    print(AStarVisitedList)

    if heuristicNum != 3:
      resetAllNodes(nodeList)
      resetHAndGValues(nodeList)

# Test case retrieval and node/edge list generation:

In [ ]:
def getTestCase():
  return input("What test file do you want to run (in the form XX)?: ")

In [ ]:
def generateNodeList(testCase):
  f = open("TestCase_" + testCase +  "_NodeID.csv")

  global nodeIDToCoordinateDict
  global startNode
  global endNode
  nodeList = []

  # Repeating code is sloppy, but I do it here as I want the start node
  # alone without having a condition repeated in the following loop.
  #
  # Extract the starting node and add it into the node list
  line = f.readline()
  nodeInfo = line.split(sep=',')
  nodeID = nodeInfo[0]
  nodeX = int(nodeInfo[1])
  nodeY = int(nodeInfo[2])
  newCoord = coordinate(nodeX, nodeY)
  newNode = node(nodeID, newCoord)

  nodeIDToCoordinateDict[newNode.id] = newCoord

  startNode = newNode

  row = [newNode]
  nodeList.append(row)

  # Keep track of this so we know when to add a new line.
  lastX = nodeX

  for line in f:
    # Extract node info from the current line
    nodeInfo = line.split(sep=',')
    nodeID = nodeInfo[0]
    nodeX = int(nodeInfo[1])
    nodeY = int(nodeInfo[2])

    # Add a new row to the node list if we have moved up in X's
    if (nodeX > lastX):
        nodeList.append([])
        lastX = nodeX

    # Add the new node to its respective x array
    newCoord = coordinate(nodeX, nodeY)
    newNode = node(nodeID, newCoord)
    nodeList[nodeX].append(newNode)

    nodeIDToCoordinateDict[newNode.id] = newCoord

  endNode = newNode

  f.close()

  return nodeList

In [ ]:
def generateEdgeList(testCase):
  f = open("TestCase_" + testCase +  "_EdgeList.txt")

  # Edge list will be a dictionary/HashMap that'll contain the node's
  # name as the key and a list of edges as the value. We'll use the dict that
  # allows nodeID -> coordinate for references in the node array.
  edgeDict = {}


  for line in f:
    # Extract edge info from the current line
    edgeInfo = line.split(sep=',')
    firstNode = edgeInfo[0]
    secondNode = edgeInfo[1]
    edgeWeight = float(edgeInfo[2])

    # Add the nodes into the edge dict if they're not in there already
    if firstNode not in edgeDict:
      edgeDict[firstNode] = []

    if secondNode not in edgeDict:
      edgeDict[secondNode] = []

    edgeDict.update(firstNode=edgeDict[firstNode].append(edge(secondNode, edgeWeight)))
    edgeDict.update(secondNode=edgeDict[secondNode].append(edge(firstNode, edgeWeight)))

  f.close()

  return edgeDict

# BFS, DFS, and A* Functions:

In [ ]:
def BFS(nodeList, edgeDict):
  visitedList = []
  global startNode
  global endNode

  # Set up the query and add the starting node to it.
  frontierQuery = deque()
  frontierQuery.append(startNode)

  while (len(frontierQuery) != 0):
    curNode = frontierQuery.popleft()
    curNode.visited = True

    visitedList.append(curNode.id)

    if curNode == endNode:
      return visitedList

    neighbors = validNeighbors(curNode, nodeList, edgeDict)

    # Luckily, none of the examples require this loop.
    # However, if there is a cycle in the graph somewhere,
    # such as 1 - 2 - 6 - 1, 6 will be added to the frontier
    # for 1 and again for 2 (if order permits). We set the neighbors
    # to visited so that it isn't picked up again by following calls
    # to neighbors.
    for neighbor in neighbors:
      neighbor.visited = True

    frontierQuery.extend(neighbors)

  return visitedList

In [ ]:
def DFS(nodeList, edgeDict):
  visitedList = []
  global startNode
  global endNode

  # Set up the stack and add the starting node to it.
  frontierStack = []

  startNode.visited = True
  frontierStack.append(startNode)

  # In lecture 6 notes, the frontier for DFS is shown by
  # adding the elements to the front and removing from
  # the front. I inverted this to achieve the same effect
  # without the costly procedure of regular list deletion
  # from the front.
  while (len(frontierStack) != 0):
    # Pop the last node added to the stack.
    curNode = frontierStack.pop()

    visitedList.append(curNode.id)

    if curNode == endNode:
      return visitedList

    neighbors = validNeighbors(curNode, nodeList, edgeDict)

    # Luckily, none of the examples require this loop.
    # However, if there is a cycle in the graph somewhere,
    # such as 1 - 2 - 6 - 1, 6 will be added to the frontier
    # for 1 and again for 2 (if order permits). We set the neighbors
    # to visited so that it isn't picked up again by following calls
    # to neighbors.
    for neighbor in neighbors:
      neighbor.visited = True

    frontierStack.extend(neighbors)

  # Set up the stack and add the starting node to it.
  return visitedList

In [ ]:
def AStar(nodeList, edgeDict, heuristicNum):
  visitedList = []

  heuristic(nodeList, heuristicNum)

  global startNode
  global endNode

  startNode.gValue = 0
  frontierQuery = deque()
  frontierQuery.append(startNode)

  while(len(frontierQuery) != 0):
    curNode = frontierQuery.popleft()
    curNode.visited = True

    visitedList.append(curNode.id)

    if curNode == endNode:
      return visitedList

    neighbors = validNeighbors(curNode, nodeList, edgeDict)
    costUpdate(curNode, neighbors, edgeDict)
    addNeighborsToFrontier(neighbors, frontierQuery)


  return visitedList

Supporting functions:

In [ ]:
def validNeighbors(curNode, nodeList, edgeDict):
  edgeList = edgeDict[curNode.id]
  neighbors = []

  # For each edge in the node's edge list, add it to neighbors
  # if it has not been visited yet.
  for edge in edgeList:
    neighborNodeID = edge.destination

    neighborNodeCoord = nodeIDToCoordinateDict[neighborNodeID]
    neighborNode = nodeList[neighborNodeCoord.x][neighborNodeCoord.y]

    if neighborNode.visited != True:
      neighbors.append(neighborNode)

  return neighbors

In [ ]:
# While this assignment only has edges of weight 1,
# this is a generic cost update function that should
# work for any weight.
def costUpdate(curNode, neighbors, edgeDict):
  edgeList = edgeDict[curNode.id]
  neighborsToRemove = []

  for neighbor in neighbors:

    # Find the edge that the neighbor connects to
    foundEdge = None

    for edge in edgeList:
      if edge.destination == neighbor.id:
        foundEdge = edge

    # Add the parent's g(n) with the weight of the edge
    # to get the child's g(n). Since we might have visited
    # this node before, we want to check if the current h(n)
    # + g(n) value is less than the update.
    newValue = curNode.gValue + foundEdge.weight

    if (newValue < neighbor.gValue):
      neighbor.gValue = newValue
    else:
      # If it is greater than, delete the node from neighbors as we
      # won't need to try to add it to the frontier.
      neighborsToRemove.append(neighbor)

  # Remove it in this loop to not cause an element in
  # the original loop to be skipped
  for neighbor in neighborsToRemove:
    neighbors.remove(neighbor)

In [ ]:
# This function allows us to add neighbors to the frontier
# in sorted order.
def addNeighborsToFrontier(neighbors, frontier):
  for neighbor in neighbors:

    try:
      neighborIndex = frontier.index(neighbor)
    except ValueError:
      frontier.append(neighbor)
      neighborIndex = len(frontier) - 1

    continueLoop = True

    # Using a boolean condition here to not clog up the while
    # loop condition with a long equation that is instead
    # executed in the loop.
    while (neighborIndex > 0 and continueLoop == True):
      nodeBeforeNeighbor = frontier[neighborIndex - 1]
      nodeBeforeNeighborValue = nodeBeforeNeighbor.gValue + nodeBeforeNeighbor.hValue

      neighborValue = neighbor.gValue + neighbor.hValue

      if (nodeBeforeNeighborValue > neighborValue):
        frontier[neighborIndex - 1] = neighbor
        frontier[neighborIndex] = nodeBeforeNeighbor

        neighborIndex -= 1
      else:
        continueLoop = False

# Heuristic Function(s):

In [ ]:
def heuristic(nodeList, heuristicNum):

  global endNode

  match heuristicNum:
    # Heuristic 1: cost of 0
    case 1:
      for row in nodeList:
        for node in row:
          node.hValue = 0.0;

      return

    # Heuristic 2: Manhattan distance
    case 2:
      endX = endNode.coordinate.x
      endY = endNode.coordinate.y

      for row in nodeList:
        for node in row:
          nodeX = node.coordinate.x
          nodeY = node.coordinate.y

          node.hValue = abs(endX - nodeX) + abs(endY - nodeY)
      return

    # Heuristic 3: Chebyshev distance
    case 3:
      endX = endNode.coordinate.x
      endY = endNode.coordinate.y

      for row in nodeList:
        for node in row:
          nodeX = node.coordinate.x
          nodeY = node.coordinate.y

          node.hValue = max(abs(endX - nodeX), abs(endY - nodeY))
      return

    case _:
      return

# Reset function

In [ ]:
def resetAllNodes(nodeList):
  for x in range(len(nodeList)):
    for y in range(len(nodeList[x])):
      nodeList[x][y].visited = False

In [ ]:
def resetHAndGValues(nodeList):
  for x in range(len(nodeList)):
    for y in range(len(nodeList[x])):
      nodeList[x][y].hValue = None
      nodeList[x][y].gValue = np.inf

# Main function call:

In [ ]:
main()

What test file do you want to run (in the form XX)?: 03
BFS: ['N_0', 'N_1', 'N_100', 'N_2', 'N_200', 'N_102', 'N_300', 'N_201', 'N_400', 'N_301', 'N_101', 'N_202', 'N_500', 'N_401', 'N_302', 'N_501', 'N_600', 'N_402', 'N_502', 'N_602', 'N_601', 'N_603', 'N_503', 'N_703', 'N_604', 'N_504', 'N_403', 'N_803', 'N_704', 'N_605', 'N_404', 'N_303', 'N_802', 'N_903', 'N_804', 'N_705', 'N_304', 'N_203', 'N_902', 'N_801', 'N_904', 'N_805', 'N_706', 'N_204', 'N_901', 'N_800', 'N_905', 'N_806', 'N_205', 'N_900', 'N_700', 'N_906', 'N_305', 'N_206', 'N_701', 'N_306', 'N_207', 'N_702', 'N_406', 'N_107', 'N_307', 'N_405', 'N_506', 'N_407', 'N_108', 'N_106', 'N_7', 'N_505', 'N_606', 'N_208', 'N_8', 'N_109', 'N_6', 'N_105', 'N_607', 'N_209', 'N_308', 'N_9', 'N_110', 'N_5', 'N_707', 'N_608', 'N_309', 'N_10', 'N_111', 'N_210', 'N_4', 'N_807', 'N_708', 'N_609', 'N_310', 'N_11', 'N_112', 'N_3', 'N_808', 'N_709', 'N_610', 'N_311', 'N_410', 'N_12', 'N_212', 'N_103', 'N_908', 'N_809', 'N_611', 'N_211', 'N_312'

# Assignment 1 Report:

# Design Choices:
**User Input:**

* User input assumes that the files exist in the content directory

**Modules**:

*   Numpy/Matplotlib: I am not experienced with either of these modules. I tried to leave the use of these to a minimum for now as I knew I could implement these algorithms without them. Numpy is only used for its definition of infinity.
*   Collections: Collections is only imported for its deque implementation for performance reasons in the BFS and A* algorithms. Deque is designed to pop from the front and append onto the end in O(1) time. These algorithms can be implemented with regular python lists, but removing values from the front takes O(n) time. It is not used to shortcut any implementation for the algorithms themselves.


**Node, Coordinate, and Edge Classes:**


*   The "node" class serves as a easy way to store all of the information that we read in. Included in the class is the node's id, x and y, if it has been visited, and the h/g value for the A* algorithm. I decided to include the h/g value here to not complicate any implementation of the A* algorithm, such as included a new class that acts as a wrapper for the node and the h/g values.
*   The "coordinate" class serves as a wrapper for the node's x and y coordinates. This was created for use in the global nodeIDToCoordinateDict.
*   The "edge" class stores the destinate and weight of the nodes. I decided to only include the destination as I store all of the edges in a dict with the source node as a key.

**Global Variables**

*   I decided to use global variables for the startNode, endNode, and a dict for quick access between these nodes and quick conversion between nodes and coordinates. Using these global variables help to not clog up method variables, too.
* The "startNode" and "endNode" global variables are used for quick access since these nodes are used in almost every algorithm and supporting method.
* The "nodeIDToCoordinateDict" is used as a way for quick conversion between nodes and coordinates in the validNeighbors. I decided to use this as I couldn't think of an easy way for the edges to access the nodes.

# Implementation Strategy:
**Sectioning:**
* The methods are separated based on their use. The sections include: main method, user input, algorithms, supporting methods, heuristic function, and reset functions.

**User Input:**
* I wanted to keep input simple, so the user only has to input the test case number.
* The generate functions read in each line, then splice them by the commas.
* The node generator function makes a node instance for each line then adds it into a 2D list.
* The edge generator makes an edge instance for each line, then adds the edges into a dict where each key is a node id and each value is a list that contains the edges from that node.

**BFS, DFS, and A Star Functions:**
* These methods implement the frontier as different collections in order to keep track of which nodes to visit next. Once a node is removed from the frontier, it is not added back.
* BFS was implemented using the deque as a queue. Nodes were inserted into the deque whenever they were found as valid neighbors. Valid neighbors were marked as visited so they are not added to the queue multiple times (which wasn't a problem for these test cases). Valid neighbors were then added to the end of the queue. This was in a while loop that repeated as long as there was a node still in the deque. Whenever we popped a node from the deque, we added its id to the visited list and returned that list if the node we popped was the end node.
* DFS was implemented with the standard python list as a stack. It follows the same procedure as BFS except we pop nodes off the end of the stack (FILO order). We don't use a deque here as the standard python list will have the same/similar performance with this procedure. Since we don't add nodes to the frontier after we remove them, there is no actual backtracking, helping improve performance as we don't have to evaluate nodes we have already evaluated.
* A* was implemented using the deque as a priority queue. We start by setting the h value for all the nodes with the heuristic function. A* follows the same procedure of BFS after this except we adjust the order of the deque based on the neighbors' g and h values in the costUpdate and addNeighborsToFrontier methods.

**Supporting Methods:**
* validNeighbors() takes the current node's edge list before looping through them to check if they have been visited. If they haven't, they are added to a list of valid neighbors that is returned once all of the edges have been evaluated.
* costUpdate() takes the neighbor list and edge list to evaluate each neighbor's g value. To find the neighbor's g value, we take the current node's g value and add the edge's weight. If the neighbor's current g value is greater than this new g value, we update it. Otherwise, we remove the neighbor from the valid neighbor list since we have a better path to it.
* addNeighborsToFrontier() attempts to add the neighbors to the frontier as a priority queue. It first finds the neighbor on the frontier or adds it to the end if the neighbor is not on the frontier already. We then shift the neighbor to the left as long as its f value (g+h) is less than the node before it on the queue. To shift, I had previously set 2 variables to copies of the node before the neighbor and the neighbor, so I set them equal to the opposite node.

**Heuristics:**
* Heuristics are implemented in a for loop that goes through each node to update the h value. The heuristic is chosen based on the heuristicNum input and they are stored in a match case. If the number inputted is not valid, the program will not be able to run since h values are None by default.

**Reset Functions:**
* These functions are a for loop that go through every node and reset their values to the default values of when they were created. The functions are split up as to only reset the h and g values whenever they are changed in the A* algorithm. This is to save computational time.

# Heuristics:

**Heuristic 1:**
* This heuristic was to set each h value as 0. I chose this as a base line to see how A* ran in comparison to the uninformed search algorithms. This heuristic is admissible since there is no h* value that will be 0 other than the goal node's. This heuristic performed exactly the same as the implementation of BFS each time and returned the exact same visited list. I believe this is because all of the edge weights are 1, so there was never any readjusting of the frontier since they are added in a BFS manner.

**Heuristic 2:**
* This heuristic was the Manhattan distance of each node to the ending node. I decided on this one since the data only allowed for us to move to an adjacent provided there was an edge there. We would never move more than one room away. This heuristic would give an h value equal to the shortest path from the node to the ending node. Since it gives the value of taking the shortest path, it is guaranteed that it is an admissible heuristic given each edge have weights of >= 1  This heuristic was the best performing heuristic in every test case. It performed better than BFS in every test case, but only did better than DFS in 1 test case. I think this is because of a low number of test cases and randomness of DFS's performance depending on which neighbor is chosen. I believe that this heuristic would do better on average than DFS if we had more test cases.

**Heuristic 3:**
* This heuristic was the Chebyshev distance of each node to the ending node. I wanted to see if a method of distance similar to Manhattan but allowing for diagonal moves would increase performance. This heuristic was admissible. Each edge has a weight of 1. One move in Chebyshev distance would be equal to either 1 or 2 edges. The shortest path would either have a weight equal to or weight lower than the true shortest path. Similar to heuristic 2, this heuristic did better than BFS in all cases but only better than DFS in 1. Unlike heuristic 2, I think this would have similar performance to DFS in future test cases.

# Comparison:

**Steps for Test Case 01:**
*   **BFS:** 25
*   **DFS:** 14
*   **A Star Heuristic 1**: 25
*   **A Star Heuristic 2**: 15
*   **A Star Heuristic 3**: 18

**Steps for Test Case 02:**

*   **BFS:** 74
*   **DFS:** 65
*   **A Star Heuristic 1**: 74
*   **A Star Heuristic 2**: 38
*   **A Star Heuristic 3**: 48

**Steps for Test Case 03:**

*   **BFS:** 977
*   **DFS:** 741
*   **A Star Heuristic 1**: 977
*   **A Star Heuristic 2**: 857
*   **A Star Heuristic 3**: 915

Above are the steps of each algorithm for each test case. We can see that BFS and A* H1 had the same number of steps. We can also see that A* H2 and H3 did better than DFS in only 1 test cases. However, an average number would be a better measure of the efficiency of the algorithms. Below is an average percentage of nodes the algorithms visited between each test case:

**Average Percentage of Nodes Visited:**
*   **BFS:** 90.57%
*   **DFS:** 65.03%
*   **A Star Heuristic 1**: 90.57%
*   **A Star Heuristic 2**: 61.23%
*   **A Star Heuristic 3**: 70.5%

If we look at these percentages, we can see that A* H2 visited on average the least amount of nodes followed by DFS and A* H3. I believe that any 3 of these algorithms would be suitable for a search algorithm, but A* H2 and A* H3 would be more scalable search algorithms.